In [472]:
from lxml import html
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import ast
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

# Leaderboard

## Leaderboard DF Generator

In [63]:
# VARIABLES
start_season = 2000
end_season = 2020

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

match_season = []
ranking = []
teams = []
matches = []
wins = []
draws = []
loss = []
goals_scored = []
goals_conceded = []
EPL_seasons = []

for i in range(start_season, end_season+1):
    EPL_seasons.append(f"{i}-{i+1}")

for season in EPL_seasons:
    url = f'https://www.worldfootball.net/schedule/eng-premier-league-{season}-spieltag/38/'
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    
    raw_final_table = soup.find_all("table", class_="standard_tabelle")[1]

    leaderboard = []
    for i in raw_final_table.find_all("td"):
        leaderboard.append(i.getText())
        
    for i in range(0, len(leaderboard)):
        if str(i).endswith('0'):
            ranking.append(leaderboard[i].strip('\n'))
        elif str(i).endswith('2'):
            teams.append(leaderboard[i].strip('\n'))
        elif str(i).endswith('3'):
            matches.append(leaderboard[i].strip('\n'))  
        elif str(i).endswith('4'):
            wins.append(leaderboard[i].strip('\n'))
        elif str(i).endswith('5'):
            draws.append(leaderboard[i].strip('\n'))
        elif str(i).endswith('6'):
            loss.append(leaderboard[i].strip('\n'))
        elif str(i).endswith('7'):
            goals_scored.append(leaderboard[i].strip('\n').split(':')[0])
            goals_conceded.append(leaderboard[i].strip('\n').split(':')[1])
    
    for _ in range(0,20):
        match_season.append(season)

In [64]:
data = {'season': match_season,
    'ranking': ranking,
       'team': teams,
       'M': matches,
       'W': wins,
       'D': draws,
       'L': loss,
       'GS': goals_scored,
       'GC': goals_conceded,
       'GD': np.nan,
       'P': np.nan}

df = pd.DataFrame(data)
for i in range(0, len(df)):
    df['GD'][i] = int(int(df['GS'][i]) - int(df['GC'][i]))
    df['P'][i] = int(df['W'][i]) * 3 + int(df['D'][i])

df['GD'] = df['GD'].astype(int)
df['P'] = df['P'].astype(int)

df.to_csv('og_leaderboard_df.csv', index=False)

## Original Leaderboard

In [458]:
df = pd.read_csv('og_leaderboard_df.csv')

In [421]:
# VARIABLES
start_season = 2000
end_season = 2020

EPL_seasons = []
for i in range(start_season, end_season+1):
    EPL_seasons.append(f"{i}-{i+1}")

for j in EPL_seasons:
    ori_season = df[df['season'] == j].drop(columns=['season', 'ranking']).reset_index(drop=True)
    ori_season.to_csv(f"./ori_results/{j}_ori.csv", index=False)

# Matches

## Match URL Generator

In [50]:
# VARIABLES
start_season = 2000
end_season = 2020

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

day = []
match_season = []
matches = []
home_team = []
away_team = []
EPL_seasons = []

for i in range(start_season, end_season+1):
    EPL_seasons.append(f"{i}-{i+1}")
    
for season in EPL_seasons:
    for matchday in range(1,39):
        url = f'https://www.worldfootball.net/schedule/eng-premier-league-{season}-spieltag/{matchday}/'
        req = requests.get(url, headers)
        soup = BeautifulSoup(req.content, 'html.parser')

        raw_table = soup.find_all("table", class_="standard_tabelle")[0]
        raw_table_a = raw_table.find_all('a')

        for i in range(0,10):
            match_season.append(season)
            day.append(matchday)
            home_team.append(raw_table_a[(i*3+0)].getText())
            away_team.append(raw_table_a[(i*3+1)].getText())
            matches.append(f"https://www.worldfootball.net{raw_table_a[(i*3+2)]['href']}")
                           
data = {'season': match_season,
       'matchday': day,
       'home_team': home_team,
       'away_team': away_team,
       'match_url': matches}

raw_match_df = pd.DataFrame(data)
raw_match_df.to_csv('raw_match_df.csv', index=False)

## Looping Through Match URLs

In [239]:
raw_match_df = pd.read_csv('raw_match_df.csv')
raw_match_df['match_stats'] = np.nan

headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [240]:
for i in range(0,len(raw_match_df)):
    url = raw_match_df['match_url'][i]
    print(url)
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    scorer_table = soup.find_all("table", class_="standard_tabelle")
    
    match_score = []
    home_goals = 0
    away_goals = 0
    goals = {}

    for j in scorer_table[1]:
        try:
            match_score.append(j.getText())
        except:
            pass

    match_score = match_score[1:]

    if len(match_score) >= 1 and match_score[0] != '\nnone\n':

        goal_counter = 0
        for k in range(0,len(match_score)):

            split_list = match_score[k].split()

            keyList = ["side", "scorer", "time_of_goal", "own_goal"]
            temp_dict = {key: None for key in keyList}

            if int(split_list[0]) > home_goals:
                temp_dict['side'] = "home"
                home_goals += 1
                goal_counter += 1

            elif int(split_list[2]) > away_goals:
                temp_dict['side'] = "away"
                away_goals += 1
                goal_counter += 1

            try:
                temp_dict['time_of_goal'] = int(split_list[4][:-1])
                temp_dict['scorer'] = f"{split_list[3]}" 
            except ValueError:
                try:
                    temp_dict['time_of_goal'] = int(split_list[5][:-1])
                    temp_dict['scorer'] = f"{split_list[3]} {split_list[4]}"
                except ValueError:
                    try:
                        temp_dict['time_of_goal'] = int(split_list[6][:-1])
                        temp_dict['scorer'] = f"{split_list[3]} {split_list[4]} {split_list[5]}" 
                    except ValueError:
                        temp_dict['time_of_goal'] = int(split_list[7][:-1])
                        temp_dict['scorer'] = f"{split_list[3]} {split_list[4]} {split_list[5]} {split_list[6]}"                    

            if 'own' in split_list:
                temp_dict['own_goal'] = True
            else:
                temp_dict['own_goal'] = False

            goals[f"{goal_counter}"] = temp_dict
    
        raw_match_df['match_stats'][i] = [goals]

https://www.worldfootball.net/report/premier-league-2000-2001-charlton-athletic-manchester-city/
https://www.worldfootball.net/report/premier-league-2000-2001-chelsea-fc-west-ham-united/
https://www.worldfootball.net/report/premier-league-2000-2001-coventry-city-middlesbrough-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-derby-county-southampton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-leeds-united-everton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-leicester-city-aston-villa/
https://www.worldfootball.net/report/premier-league-2000-2001-liverpool-fc-bradford-city/
https://www.worldfootball.net/report/premier-league-2000-2001-sunderland-afc-arsenal-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-tottenham-hotspur-ipswich-town/
https://www.worldfootball.net/report/premier-league-2000-2001-manchester-united-newcastle-united/
https://www.worldfootball.net/report/premier-league-2000-2001-arsenal-fc-liverpo

https://www.worldfootball.net/report/premier-league-2000-2001-manchester-united-leeds-united/
https://www.worldfootball.net/report/premier-league-2000-2001-newcastle-united-everton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-tottenham-hotspur-derby-county/
https://www.worldfootball.net/report/premier-league-2000-2001-west-ham-united-arsenal-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-bradford-city-ipswich-town/
https://www.worldfootball.net/report/premier-league-2000-2001-charlton-athletic-middlesbrough-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-chelsea-fc-coventry-city/
https://www.worldfootball.net/report/premier-league-2000-2001-liverpool-fc-leicester-city/
https://www.worldfootball.net/report/premier-league-2000-2001-aston-villa-sunderland-afc/
https://www.worldfootball.net/report/premier-league-2000-2001-southampton-fc-manchester-city/
https://www.worldfootball.net/report/premier-league-2000-2001-chelsea-fc-tottenh

https://www.worldfootball.net/report/premier-league-2000-2001-coventry-city-southampton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-liverpool-fc-arsenal-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-manchester-united-ipswich-town/
https://www.worldfootball.net/report/premier-league-2000-2001-sunderland-afc-manchester-city/
https://www.worldfootball.net/report/premier-league-2000-2001-tottenham-hotspur-middlesbrough-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-derby-county-newcastle-united/
https://www.worldfootball.net/report/premier-league-2000-2001-leeds-united-aston-villa/
https://www.worldfootball.net/report/premier-league-2000-2001-leicester-city-west-ham-united/
https://www.worldfootball.net/report/premier-league-2000-2001-charlton-athletic-everton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-chelsea-fc-bradford-city/
https://www.worldfootball.net/report/premier-league-2000-2001-arsenal-fc-leicest

https://www.worldfootball.net/report/premier-league-2000-2001-bradford-city-west-ham-united/
https://www.worldfootball.net/report/premier-league-2000-2001-coventry-city-charlton-athletic/
https://www.worldfootball.net/report/premier-league-2000-2001-derby-county-aston-villa/
https://www.worldfootball.net/report/premier-league-2000-2001-ipswich-town-everton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-leicester-city-sunderland-afc/
https://www.worldfootball.net/report/premier-league-2000-2001-middlesbrough-fc-southampton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-newcastle-united-manchester-city/
https://www.worldfootball.net/report/premier-league-2000-2001-tottenham-hotspur-leeds-united/
https://www.worldfootball.net/report/premier-league-2000-2001-manchester-united-arsenal-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-liverpool-fc-chelsea-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-leeds-united-manch

https://www.worldfootball.net/report/premier-league-2000-2001-arsenal-fc-leeds-united/
https://www.worldfootball.net/report/premier-league-2000-2001-aston-villa-coventry-city/
https://www.worldfootball.net/report/premier-league-2000-2001-bradford-city-middlesbrough-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-chelsea-fc-everton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-leicester-city-tottenham-hotspur/
https://www.worldfootball.net/report/premier-league-2000-2001-liverpool-fc-newcastle-united/
https://www.worldfootball.net/report/premier-league-2000-2001-manchester-united-derby-county/
https://www.worldfootball.net/report/premier-league-2000-2001-sunderland-afc-charlton-athletic/
https://www.worldfootball.net/report/premier-league-2000-2001-west-ham-united-southampton-fc/
https://www.worldfootball.net/report/premier-league-2000-2001-ipswich-town-manchester-city/
https://www.worldfootball.net/report/premier-league-2000-2001-charlton-athletic-

...

https://www.worldfootball.net/report/premier-league-2019-2020-watford-fc-manchester-city/
https://www.worldfootball.net/report/premier-league-2019-2020-aston-villa-arsenal-fc/
https://www.worldfootball.net/report/premier-league-2019-2020-manchester-united-west-ham-united/
https://www.worldfootball.net/report/premier-league-2019-2020-liverpool-fc-chelsea-fc/
https://www.worldfootball.net/report/premier-league-2019-2020-arsenal-fc-watford-fc/
https://www.worldfootball.net/report/premier-league-2019-2020-burnley-fc-brighton-hove-albion/
https://www.worldfootball.net/report/premier-league-2019-2020-chelsea-fc-wolverhampton-wanderers/
https://www.worldfootball.net/report/premier-league-2019-2020-crystal-palace-tottenham-hotspur/
https://www.worldfootball.net/report/premier-league-2019-2020-everton-fc-afc-bournemouth/
https://www.worldfootball.net/report/premier-league-2019-2020-leicester-city-manchester-united/
https://www.worldfootball.net/report/premier-league-2019-2020-manchester-city-no

https://www.worldfootball.net/report/premier-league-2020-2021-chelsea-fc-sheffield-united/
https://www.worldfootball.net/report/premier-league-2020-2021-west-ham-united-fulham-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-west-bromwich-albion-tottenham-hotspur/
https://www.worldfootball.net/report/premier-league-2020-2021-leicester-city-wolverhampton-wanderers/
https://www.worldfootball.net/report/premier-league-2020-2021-manchester-city-liverpool-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-arsenal-fc-aston-villa/
https://www.worldfootball.net/report/premier-league-2020-2021-newcastle-united-chelsea-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-aston-villa-brighton-hove-albion/
https://www.worldfootball.net/report/premier-league-2020-2021-tottenham-hotspur-manchester-city/
https://www.worldfootball.net/report/premier-league-2020-2021-manchester-united-west-bromwich-albion/
https://www.worldfootball.net/report/premier-league-

https://www.worldfootball.net/report/premier-league-2020-2021-tottenham-hotspur-leeds-united/
https://www.worldfootball.net/report/premier-league-2020-2021-crystal-palace-sheffield-united/
https://www.worldfootball.net/report/premier-league-2020-2021-brighton-hove-albion-wolverhampton-wanderers/
https://www.worldfootball.net/report/premier-league-2020-2021-west-bromwich-albion-arsenal-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-newcastle-united-leicester-city/
https://www.worldfootball.net/report/premier-league-2020-2021-chelsea-fc-manchester-city/
https://www.worldfootball.net/report/premier-league-2020-2021-southampton-fc-liverpool-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-burnley-fc-fulham-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-sheffield-united-newcastle-united/
https://www.worldfootball.net/report/premier-league-2020-2021-wolverhampton-wanderers-everton-fc/
https://www.worldfootball.net/report/premier-league-2

https://www.worldfootball.net/report/premier-league-2020-2021-manchester-city-west-ham-united/
https://www.worldfootball.net/report/premier-league-2020-2021-west-bromwich-albion-brighton-hove-albion/
https://www.worldfootball.net/report/premier-league-2020-2021-leeds-united-aston-villa/
https://www.worldfootball.net/report/premier-league-2020-2021-newcastle-united-wolverhampton-wanderers/
https://www.worldfootball.net/report/premier-league-2020-2021-crystal-palace-fulham-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-leicester-city-arsenal-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-tottenham-hotspur-burnley-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-chelsea-fc-manchester-united/
https://www.worldfootball.net/report/premier-league-2020-2021-sheffield-united-liverpool-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-everton-fc-southampton-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-bur

https://www.worldfootball.net/report/premier-league-2020-2021-burnley-fc-west-ham-united/
https://www.worldfootball.net/report/premier-league-2020-2021-manchester-united-liverpool-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-leicester-city-newcastle-united/
https://www.worldfootball.net/report/premier-league-2020-2021-leeds-united-tottenham-hotspur/
https://www.worldfootball.net/report/premier-league-2020-2021-sheffield-united-crystal-palace/
https://www.worldfootball.net/report/premier-league-2020-2021-manchester-city-chelsea-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-liverpool-fc-southampton-fc/
https://www.worldfootball.net/report/premier-league-2020-2021-wolverhampton-wanderers-brighton-hove-albion/
https://www.worldfootball.net/report/premier-league-2020-2021-aston-villa-manchester-united/
https://www.worldfootball.net/report/premier-league-2020-2021-west-ham-united-everton-fc/
https://www.worldfootball.net/report/premier-league-2020-202

In [241]:
raw_match_df.to_csv("match_df.csv", index=False)

## Replaying The Matches

In [455]:
match_df = pd.read_csv("match_df.csv")
match_df['60_min_home_goals'] = np.nan
match_df['60_min_away_goals'] = np.nan
match_df['60_min_goal_scorers'] = np.nan

<img src='average_stoppage_time.PNG'>

We will assume total average stoppage time per match to be 8 mins, which is +4 mins per half. 

Hence, we will only take goals between:
- **0th to 34th minute-mark**
- **45th to 79th minute-mark.**

In [456]:
for i in range(0, len(match_df)):
    home_goals = 0
    away_goals = 0
    scorer_list = []

    if type(match_df['match_stats'][i]) == str:
        try:
            stats_dict = ast.literal_eval(match_df['match_stats'][i])[0]
            
            for j in range(1, len(stats_dict)+1):
                condition = False
                if stats_dict[f"{j}"]['time_of_goal'] <= 34:
                    condition = True
                elif stats_dict[f"{j}"]['time_of_goal'] >= 45 and stats_dict[f"{j}"]['time_of_goal'] <= 79:
                    condition = True

                if condition == True:
                    if stats_dict[f"{j}"]['side'] == 'home':
                        home_goals += 1
                    elif stats_dict[f"{j}"]['side'] == 'away':
                        away_goals += 1

                if condition == True and stats_dict[f"{j}"]['own_goal'] == False:
                    scorer_list.append(stats_dict[f"{j}"]['scorer'])
        except:
            pass

    match_df['60_min_home_goals'][i] = home_goals
    match_df['60_min_away_goals'][i] = away_goals
    match_df['60_min_goal_scorers'][i] = scorer_list

In [457]:
# VARIABLES
start_season = 2000
end_season = 2020

EPL_seasons = []
for i in range(start_season, end_season+1):
    EPL_seasons.append(f"{i}-{i+1}")

for j in EPL_seasons:
    season_matches = match_df[match_df['season'] == j]
    season_matches.reset_index(drop=True, inplace=True)

    season_dict = {}
    for i in season_matches['home_team'].unique():
        season_dict[i] = {'W': 0, 'D': 0, 'L': 0, 'GS': 0, 'GC': 0}
    
    for i in range(0, len(season_matches)):
        if season_matches['60_min_home_goals'][i] > season_matches['60_min_away_goals'][i]:
            season_dict[season_matches['home_team'][i]]['W'] += 1
            season_dict[season_matches['away_team'][i]]['L'] += 1
        elif season_matches['60_min_away_goals'][i] > season_matches['60_min_home_goals'][i]:
            season_dict[season_matches['home_team'][i]]['L'] += 1
            season_dict[season_matches['away_team'][i]]['W'] += 1
        elif season_matches['60_min_away_goals'][i] == season_matches['60_min_home_goals'][i]:
            season_dict[season_matches['home_team'][i]]['D'] += 1
            season_dict[season_matches['away_team'][i]]['D'] += 1

        season_dict[season_matches['home_team'][i]]['GS'] += season_matches['60_min_home_goals'][i]
        season_dict[season_matches['home_team'][i]]['GC'] += season_matches['60_min_away_goals'][i]
        season_dict[season_matches['away_team'][i]]['GS'] += season_matches['60_min_away_goals'][i]
        season_dict[season_matches['away_team'][i]]['GC'] += season_matches['60_min_home_goals'][i]

    season_df = pd.DataFrame.from_dict(season_dict,orient='index')
    season_df['GD'] = season_df['GS'] - season_df['GC']
    season_df['P'] = season_df['W']*3 + season_df['D']
    season_df.sort_values('P', ascending=False, inplace=True)
    season_df.reset_index(inplace=True)
    season_df.rename(columns={"index": "team"}, inplace=True)
    season_df.to_csv(f"./60min_results/{j}_60min.csv", index=False)
    
    
    scorer_list = [] 

    for i in season_matches['60_min_goal_scorers']:
        scorer_list.extend(i)

    scorer_dict = dict.fromkeys(set(scorer_list), 0)

    for i in scorer_list:
        scorer_dict[i] += 1

    scorer_df = pd.DataFrame.from_dict(scorer_dict, orient='index').reset_index()
    scorer_df.rename(columns={'index': 'player', 0: 'goals'}, inplace=True)
    scorer_df.sort_values('goals', ascending = False, inplace=True)
    scorer_df.reset_index(drop=True, inplace=True)
    scorer_df.to_csv(f"./60min_scorer/{j}_topscorer_60min.csv", index=False)

# Comparison

## Premier League Winners for the past 21 seasons

**90-Minute Winning Teams**

In [490]:
df = pd.read_csv('og_leaderboard_df.csv')
df[df['ranking'] == 1]['team'].value_counts()

Manchester United    7
Chelsea FC           5
Manchester City      5
Arsenal FC           2
Leicester City       1
Liverpool FC         1
Name: team, dtype: int64

**60-Minute Winning Teams**

In [489]:
# VARIABLES
start_season = 2000
end_season = 2020

EPL_seasons = []
for i in range(start_season, end_season+1):
    EPL_seasons.append(f"{i}-{i+1}")

winner_list = []
for i in EPL_seasons:
    df = pd.read_csv(f"./60min_results/{i}_60min.csv", index_col=None)
    winner_list.append(df['team'][0])

Counter(winner_list)

Counter({'Manchester United': 7,
         'Arsenal FC': 4,
         'Chelsea FC': 4,
         'Manchester City': 4,
         'Tottenham Hotspur': 1,
         'Liverpool FC': 1})

## 2020-2021 Season Case Study

In [491]:
df = pd.read_csv('og_leaderboard_df.csv')
df_20_21 = df[df['season'] == '2020-2021']

df_60min = pd.read_csv('./60min_results/2020-2021_60min.csv')

**Original Leaderboard**

In [512]:
df_20_21.drop(columns=['M'])

,season,ranking,team,W,D,L,GS,GC,GD,P
400,2020-2021,1,Manchester City,27,5,6,83,32,51,86
401,2020-2021,2,Manchester United,21,11,6,73,44,29,74
402,2020-2021,3,Liverpool FC,20,9,9,68,42,26,69
403,2020-2021,4,Chelsea FC,19,10,9,58,36,22,67
404,2020-2021,5,Leicester City,20,6,12,68,50,18,66
405,2020-2021,6,West Ham United,19,8,11,62,47,15,65
406,2020-2021,7,Tottenham Hotspur,18,8,12,68,45,23,62
407,2020-2021,8,Arsenal FC,18,7,13,55,39,16,61
408,2020-2021,9,Leeds United,18,5,15,62,54,8,59
409,2020-2021,10,Everton FC,17,8,13,47,48,-1,59


**60 minute Leaderboard**

In [514]:
df_temp = df_60min.reset_index().rename(columns = {'index': 'ranking'})
for i in range(0,len(df_temp)):
    df_temp['ranking'][i] += 1
df_temp['GS'] = df_temp['GS'].astype(int)
df_temp['GC'] = df_temp['GC'].astype(int)
df_temp['GD'] = df_temp['GD'].astype(int)
df_temp

,ranking,team,W,D,L,GS,GC,GD,P
0,1,Manchester City,25,7,6,67,23,44,82
1,2,Liverpool FC,20,11,7,47,27,20,71
2,3,Tottenham Hotspur,20,9,9,57,31,26,69
3,4,West Ham United,18,12,8,40,29,11,66
4,5,Manchester United,17,14,7,52,31,21,65
5,6,Chelsea FC,16,14,8,44,27,17,62
6,7,Leicester City,15,11,12,43,37,6,56
7,8,Arsenal FC,15,9,14,38,32,6,54
8,9,Everton FC,14,11,13,38,38,0,53
9,10,Leeds United,13,11,14,44,43,1,50


**IQR Comparison**

In [524]:
print(df_20_21['P'].std())
print(df_60min['P'].std())

16.887475895031297
14.085733730865499


**Goal Comparison**

In [497]:
print(f"Normal:{sum(df_20_21['GS'])}")
print(f"60 Min:{sum(df_60min['GS'])}")

Normal:1024
60 Min:749.0


In [529]:
df_scorer = pd.read_csv("./60min_scorer/2020-2021_topscorer_60min.csv", index_col=False)

In [535]:
df_top10_scorer = df_scorer.head(10)
df_top10_scorer.reset_index(inplace=True)
ranking_list = ['1', '2', 'T3', 'T3', '5', 'T6', 'T6', '8', 'T9', 'T9']
df_top10_scorer['index'] = ranking_list
df_top10_scorer

,index,player,goals
0,1,Harry Kane,19
1,2,Mohamed Salah,17
2,T3,Patrick Bamford,14
3,T3,Bruno Fernandes,14
4,5,Heung-min Son,13
5,T6,Dominic Calvert-Lewin,12
6,T6,İlkay Gündoğan,12
7,8,Matheus Pereira,11
8,T9,Wilfried Zaha,10
9,T9,Alexandre Lacazette,10


In [536]:
list1 = ['1', '2', '3', 'T4', 'T4', '6', '7', '8', 'T9', 'T9']
list2 = ['Harry Kane', 'Mohamed Salah', 'Bruno Fernandes', 'Patrick Bamford', 'Heung-min Son', 'Dominic Calvert-Lewin', 'Jamie Vardy', 'Ollie Watkins', 'İlkay Gündoğan', 'Alexandre Lacazette']
list3 = [23, 22, 18, 17, 17, 16, 15, 14, 13, 13]

d = {'index': list1, 'player': list2, 'goals': list3}

pd.DataFrame(d)

,index,player,goals
0,1,Harry Kane,23
1,2,Mohamed Salah,22
2,3,Bruno Fernandes,18
3,T4,Patrick Bamford,17
4,T4,Heung-min Son,17
5,6,Dominic Calvert-Lewin,16
6,7,Jamie Vardy,15
7,8,Ollie Watkins,14
8,T9,İlkay Gündoğan,13
9,T9,Alexandre Lacazette,13
